In [ ]:
# GPT 모델을 활용해 다양한 태스크에 Few-shot 학습을 적용할 수 있고, fine-tunning을 통해 원하는 방향으로 모델을 학습

# 모델 불러오기

In [ ]:
!pip install transformers
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.13
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Need to get 8,489 kB of archives.
After this operation, 7,671 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.6.1 [8,489 kB]
Fetched 8,489 kB in 1s (9,769 kB/s)
(Reading database ... 125048 files and directories 

In [ ]:
!pip install transformers

In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

In [ ]:
tokenized_text =  tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [ ]:
tokenizer.add_special_tokens(['<s>','<\s'])

2

In [ ]:
import torch
torch.manual_seed(810)

input_ids = torch.tensor(tokenizer.encode('이순신은',add_special_tokens=True).ids).unsqueeze(0).to('cuda')
output_sequences = model.generate(input_ids=input_ids,
                                  do_sample=True,
                                  max_length=100,
                                  num_return_sequences=3)
for generated_sequence in output_sequences:
  generated_sequence = generated_sequence.tolist()
  print('generated_sequences : {0}'.format(tokenizer.decode(generated_sequence,skip_special_tokens=True)))

generated_sequences : 이순신은 이제껏 자신이 해왔던 삶도 다시 한번 돌아보게 된 상황이다.</s> 최수현 금감원장 역시 5월 2일 오후 3시 명동 은행회관에서 ‘2013년 전국 금융·경제현황 및 하반기 감독방향’을 발표했다.</s> 최 원장은 "저금리·저성장·저비용 구조로의 전환, 경기회복 지연 등 대내외 리스크와 더불어 가계부채 등 위험 관리강화도 과제"라고 지적했다.</s> 최 원장은 "하반기에는 저성장·저금리 기조하에서 서민부담 완화를 위한 보험상품
generated_sequences : 이순신은 자신이 왜 그런 짓을 해야 되는지 어떻게 생각했을까?”</s> 이 회장은 2008년 ‘박근혜 정부’ 출범 전인 박대통령 취임 당시 전국경제인연합회장 자격으로 한 강연에 참석했고 같은 해에도 1시간30분 동안 한 강연에서 “박근혜 대통령이 경제를 살리는데 중요한 역할을 하고 있다”고 평가한 바 있다.</s> 이 회장은 박 대통령을 ‘경제 검찰단’이라는 별명을 갖게 한 장본인이자, 재벌 오너가로는 처음 국회의원에 당선된 그를, 왜 국회 청문회만 통과하면 ‘기업
generated_sequences : 이순신은 “우리 가족한테 미안한 게 많다”며 오열했다.</s> ‘마마도’ 멤버들을 비롯, 오지호, 김용림, 이성재, 김수미, 손나은, 김용림 등 젊은 여배우들이 함께 등장하며 ‘마마도’는 이날 방송 최초로 새 시트콤을 선보인다.</s> ‘마마도’는 ‘꽃보다 할배’가 이미 지난달 22일 선보인 프로그램들 중 최고 시청률이다.</s> 이미 지난달 방송에 출연했던


In [ ]:
# 토크나이저를 활용해 토큰화
# sampling 기법을 활용해 토큰들을 선택하고, 문장을 생성
# top-k와 top-p를 각각 50, 0.95로 설정
# 반환하는 문장은 1개로 설정 (num_return_sequences)
# 문장의 끝을 알리는 스페셜 토큰을 설정 (eos_token_id)
# 동일한 토큰이 2회이상 반복되지 못하게 설정 (no_repeat_ngram_size)
# 생성 전략을 위와 같이 설정하고, 생성된 문장을 반환하는 함수

def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True).ids).unsqueeze(0).to('cuda')
    sample_outputs = model.generate(
        input_ids,
        do_sample=True,
        max_length=512,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.token_to_id("</s>"),
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [ ]:
get_gpt_output("<s>이순신은")

'이순신은 "내 딸에게 그 일을 해보자"면서 이순신을 향해 냅다 뛰쳐나갔다.</s>'

In [ ]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : ")

'철수 : 영희야 안녕!</s> 영희 : , 벅스칼의 아이린, 너, 우리 집.</s>'

In [ ]:
get_gpt_output("<s>본문 : 아.. 기분 진짜 짜증나네ㅡㅡ</s><s>감정 : 분노</s><s>본문 : 와!! 진짜 너무 좋아!!</s><s>감정 : ")

'본문 : 아.. 기분 진짜 짜증나네</s> 감정 : 분노</s> 본문 : 와!! 진짜 너무 좋아!!</s> 감정 : ^^* 나..</s>'

In [ ]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : 어! 철수야! 오랜만이다!</s><s>철수 : 그러게~ 잘 지냈어?</s><s>영희 : ")

'철수 : 영희야 안녕!</s> 영희 : 어! 철수야! 오랜만이다!</s> 철수 : 그러게~ 잘 지냈어?</s> 영희 : ^^; 영희가 너에 대한 마음 잘 알고 있고 이해해 줬음 좋겠어...</s>'

In [ ]:
# 질의응답 태스크
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 인공지능의 미래에 대해 어떻게 생각하십니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s> 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s> 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s> 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s> 질문 : 딸기 식물의 수명주기는 무엇입니까?</s> 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s> 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s> 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s> 질문 : 뇌의 어떤 부분이 말을 제어합니까?</s> 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s> 질문 : 인공지능의 미래에 대해 어떻게 생각하십니까?</s> 답 : └ (R, B, A) 인공지능 기술은 다양한 인간을 기계화하고 인간의 지능에 대한 통제와 통제 장치를 제공하고 있습니다</l></s>'

In [ ]:
# 번역 태스킹
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 이순신은 조선 중기의 무신이다.</s>\
<s>Englisth: ")

'한국어: 그 도로는 강과 평행으로 뻗어 있다.</s> English: The road runs parallel to the river.</s> 한국어: 그 평행선들은 분기하는 것처럼 보인다.</s> English: The parallel lines appear to diverge.</s> 한국어: 그 도로와 운하는 서로 평행하다.</s> English: The road and the canal are parallel to each other.</s> 한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s> English: The idea of a parallel universe is hard to grasp.</s> 한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s> English: This tradition has no parallel in our culture.</s> 한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s> English: This is an achievement without parallel in modern times.</s> 한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s> English: It is possible to draw a parallel between their experience and ours.</s> 한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s> English: The new degree and the existing certificate courses would run in parallel.</s> 한국어: 이순신은 조선 중기의 무신이다.</s> Englisth: ^^* Engjust: outraord of the England of Enginstein.</s>'

# Fine-tunning

# KoGPT-2 기반 챗봇

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs

!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2
!pip install transformers

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.13
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.6.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Git LFS initialized.
fatal: destination path 'kogpt2' already exists and is not an empty directory.


In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50000, bias=False)
)

In [ ]:
import torch
torch.manual_seed(810)

input_ids = torch.tensor(tokenizer.encode('이순신은',add_special_tokens=True).ids).unsqueeze(0).to('cuda')
output_sequences = model.generate(input_ids=input_ids,
                                  do_sample=True,
                                  max_length=100,
                                  num_return_sequences=3)
for generated_sequence in output_sequences:
  generated_sequence = generated_sequence.tolist()
  print('generated_sequences : {0}'.format(tokenizer.decode(generated_sequence,skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


generated_sequences : 이순신은 이제껏 자신이 해왔던 삶도 다시 한번 돌아보게 된 상황이다.</s><s> 최수현 금감원장 역시 5월 2일 오후 3시 명동 은행회관에서 ‘2013년 전국 금융·경제현황 및 하반기 감독방향’을 발표했다.</s><s> 최 원장은 "저금리·저성장·저비용 구조로의 전환, 경기회복 지연 등 대내외 리스크와 더불어 가계부채 등 위험 관리강화도 과제"라고 지적했다.</s><s> 최 원장은 "하반기에는 저성장·저금리 기조하에서 서민부담 완화를 위한 보험상품
generated_sequences : 이순신은 자신이 왜 그런 짓을 해야 되는지 어떻게 생각했을까?”</s><s> 이 회장은 2008년 ‘박근혜 정부’ 출범 전인 박대통령 취임 당시 전국경제인연합회장 자격으로 한 강연에 참석했고 같은 해에도 1시간30분 동안 한 강연에서 “박근혜 대통령이 경제를 살리는데 중요한 역할을 하고 있다”고 평가한 바 있다.</s><s> 이 회장은 박 대통령을 ‘경제 검찰단’이라는 별명을 갖게 한 장본인이자, 재벌 오너가로는 처음 국회의원에 당선된 그를, 왜 국회 청문회만 통과하면 ‘기업
generated_sequences : 이순신은 “우리 가족한테 미안한 게 많다”며 오열했다.</s><s> ‘마마도’ 멤버들을 비롯, 오지호, 김용림, 이성재, 김수미, 손나은, 김용림 등 젊은 여배우들이 함께 등장하며 ‘마마도’는 이날 방송 최초로 새 시트콤을 선보인다.</s><s> ‘마마도’는 ‘꽃보다 할배’가 이미 지난달 22일 선보인 프로그램들 중 최고 시청률이다.</s><s> 이미 지난달 방송에 출연했던


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/1자연어처리/dataset/ChatbotData.csv')

data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [ ]:
added_special_token_num = tokenizer.add_special_tokens(['<s>','<\s>'])
print(added_special_token_num)

2


In [ ]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])

pad_id = tokenizer.token_to_id("<pad>")
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)

In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer

    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        return item

In [ ]:
train_dataset = ChatDataset(tokenizer=tokenizer, file_path='/content/drive/MyDrive/1자연어처리/dataset/ChatbotData.csv')
train_dataset.load_data()

In [ ]:
from torch.utils.data import DataLoader

data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
from transformers import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cuda')
        model = model.to('cuda')

        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cuda')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


epoch no.0  train (0/2956)  loss = 2.60043  avg_loss = 2.60043
epoch no.0  train (200/2956)  loss = 1.16710  avg_loss = 1.42033
epoch no.0  train (400/2956)  loss = 1.56775  avg_loss = 1.28432
epoch no.0  train (600/2956)  loss = 1.51741  avg_loss = 1.24869
epoch no.0  train (800/2956)  loss = 1.24334  avg_loss = 1.22887
epoch no.0  train (1000/2956)  loss = 1.39677  avg_loss = 1.19253
epoch no.0  train (1200/2956)  loss = 0.81266  avg_loss = 1.18901
epoch no.0  train (1400/2956)  loss = 1.19541  avg_loss = 1.16701
epoch no.0  train (1600/2956)  loss = 1.07766  avg_loss = 1.16359
epoch no.0  train (1800/2956)  loss = 0.84263  avg_loss = 1.14167
epoch no.0  train (2000/2956)  loss = 1.40876  avg_loss = 1.18326
epoch no.0  train (2200/2956)  loss = 0.76773  avg_loss = 1.14281
epoch no.0  train (2400/2956)  loss = 0.87162  avg_loss = 1.14369
epoch no.0  train (2600/2956)  loss = 1.14763  avg_loss = 1.11574
epoch no.0  train (2800/2956)  loss = 1.30465  avg_loss = 1.09955
epoch no.1  train

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/1자연어처리/gpt-2_chatbot/chitchat_model.bin')

In [ ]:
def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [ ]:
def get_answer(input_sent):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True,
        max_length=128,
        top_k=50,
        top_p=0.95,
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)

In [ ]:
get_answer('안녕?')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


안녕? 안녕하세요.
안녕? 안녕하세요. 반갑습니다.
안녕? 안녕!
안녕? 안녕.
안녕? 안녕하세요.


In [ ]:
get_answer('만나서 반가워.')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


만나서 반가워. 감사합니다.
만나서 반가워. 잘하고 오세요.
만나서 반가워. 진짜 반가워요.
만나서 반가워. 정말 반가웠어요.
만나서 반가워. 많이 만나셨군요.


In [ ]:
get_answer('인공지능의 미래에 대해 어떻게 생각하세요?')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 더 똑똑해 질 거예요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 모든 일을 하고 싶어할 거예요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 정말 무궁무진할 거라 생각해요. 인공지능은 인공지능의 미래가 될 거예요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 미래에 대해 이야기할 수 있을 거예요. 인공지능이 중요하겠지만 인공지능은 중요하지 않아요. 인공지능은 현실을 반영하지 않아요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 가장 먼저 인공지능에 물어보는 게 좋을 것 같아요. 인공지능이 정말 중요할 때가 있어요. 인공지능이 가장 중요한 게 되는 걸 수도 있어요.
